<a href="https://colab.research.google.com/github/Shawty2084/CSE523-Machine-Learning-2022-Mandelbrot_Set/blob/main/SVM%2BLDA_Mid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import glob
import os

In [2]:
dir = os.chdir("/content/drive/MyDrive/Colab Notebooks/BBC_Data")
print(f'Directory change to {os.getcwd()}')

Directory change to /content/drive/MyDrive/Colab Notebooks/BBC_Data


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def import_data(folders_n, def_path):
  filename = [path + "/News Articles/" + folder + "/" + "*.txt"
     for path,folder in zip([def_path]*len(folders_n), folders_n)]
  temp_doc = []
  print(filename)
  # tags 
  for x in filename:
    data = glob.glob(x) 
    for y in data:
      with open(y, encoding="latin-1") as f:
        topic = x.split('/')[len(x.split('/'))-2] 
        # print('me',topic)
        line = f.readlines()
        # head = line[0].strip()
        body = ''.join([l.strip() for l in line[1:]])
        temp_doc.append([topic,body])
    print(f"Loading data from \033[1m{topic}\033[0m directory")
    print("\nEntire Data is loaded successfully")

  return temp_doc

In [5]:
folders_n = ['business','entertainment','politics','sport','tech']
doc = import_data(folders_n = folders_n, def_path = os.getcwd())
# docs = load_data(folder_names = folders_n, root_path = os.getcwd())


['/content/drive/My Drive/Colab Notebooks/BBC_Data/News Articles/business/*.txt', '/content/drive/My Drive/Colab Notebooks/BBC_Data/News Articles/entertainment/*.txt', '/content/drive/My Drive/Colab Notebooks/BBC_Data/News Articles/politics/*.txt', '/content/drive/My Drive/Colab Notebooks/BBC_Data/News Articles/sport/*.txt', '/content/drive/My Drive/Colab Notebooks/BBC_Data/News Articles/tech/*.txt']
Loading data from business directory

Entire Data is loaded successfully
Loading data from entertainment directory

Entire Data is loaded successfully
Loading data from politics directory

Entire Data is loaded successfully
Loading data from sport directory

Entire Data is loaded successfully
Loading data from tech directory

Entire Data is loaded successfully


In [6]:
# print(type(doc))
doc1 = pd.DataFrame(doc, columns = ['category','text'])
length = len(doc1)
# print(length)
doc1.head(length)
# print(doc1)

,category,text
0,business,Ethiopia produced 14.27 million tonnes of crop...
1,business,Ask Jeeves has become the third leading online...
2,business,Budget airline Ryanair has placed an order for...
3,business,Crude oil prices surged back above the $47 a b...
4,business,Revenues at media group Reuters slipped 11% du...
...,...,...
2220,tech,Now that a critical mass of people have embrac...
2221,tech,Video games on consoles and computers proved m...
2222,tech,Mobile phones are still enjoying a boom time i...
2223,tech,Music and film fans will be able to control th...


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# download necessary resources for NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)
    return text

doc1['text_processed'] = doc1['text'].apply(preprocess_text)
print(doc1['text'])


0       Ethiopia produced 14.27 million tonnes of crop...
1       Ask Jeeves has become the third leading online...
2       Budget airline Ryanair has placed an order for...
3       Crude oil prices surged back above the $47 a b...
4       Revenues at media group Reuters slipped 11% du...
                              ...                        
2220    Now that a critical mass of people have embrac...
2221    Video games on consoles and computers proved m...
2222    Mobile phones are still enjoying a boom time i...
2223    Music and film fans will be able to control th...
2224    Cyberspace is becoming a very popular destinat...
Name: text, Length: 2225, dtype: object


In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [10]:
df = doc1
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['category'], test_size=0.2, random_state=42)
print("X_train",X_train.head(2), "\n")
print("X_test",X_test.head(2), "\n")
print("y_train", y_train.head(2), "\n")
print("y_test", y_test.head(2), "\n")
print((X_train), X_test, y_train, y_test)


X_train 1490    New Southampton manager Harry Redknapp faces a...
2001    A new European directive could put software wr...
Name: text, dtype: object 

X_test 414    A former FBI agent and an internet stock picke...
420    India has signed a $40bn (Â£21bn) deal to impo...
Name: text, dtype: object 

y_train 1490    sport
2001     tech
Name: category, dtype: object 

y_test 414    business
420    business
Name: category, dtype: object 

1490    New Southampton manager Harry Redknapp faces a...
2001    A new European directive could put software wr...
1572    Joint assistant boss Stuart Pearce has admitte...
1840    Sony's Playstation Portable is the top gadget ...
610     The late US legend Elvis Presley is likely to ...
                              ...                        
1638    Bolton boss Sam Allardyce has signed Roma defe...
1095    A "taxpayer subsidised propaganda exercise" on...
1130    Three Labour councillors in Birmingham were ca...
1294    People vulnerable to problem d

In [11]:

vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print(vectorizer)

CountVectorizer(stop_words='english')


In [12]:
lda = LatentDirichletAllocation(n_components=5, random_state=42)
X_train_lda = lda.fit_transform(X_train)
X_test_lda = lda.transform(X_test)
# print(type(X_train_lda))
print(len(X_train_lda), len(X_test_lda))

1780 445


In [13]:
from sklearn.svm import SVC
def svm_model(kernel, C):
  svm = SVC(kernel=kernel, C=C)
  # print(C)
  # if C ==300:
    # print(C)
  svm.fit(X_train_lda, y_train)
  return(svm)
  


In [14]:
kernel_mod = ['linear', 'sigmoid', 'rbf', 'poly']
accuracy_list = []
max_acc, max_k, max_C = 0, 0, 0

In [15]:
for k in kernel_mod:
  # print(k)
  for c_range in range(1,300):
    if k == 'poly' and c_range == 26:
      break
    svm = svm_model(k, c_range)
    # svm.fit(X_train_lda, y_train)
    y_pred = svm.predict(X_test_lda)
    accuracy = accuracy_score(y_test, y_pred)
    max_acc = max(max_acc, accuracy)
    if max_acc == accuracy:
      max_k = k
      max_C = c_range
  accuracy_list.append(max_acc)
    # accuracy_list.append(accuracy)
  # print("Accuracy:", accuracy)
# print(y_pred)
print(accuracy_list)
# print(len(y_pred))

[0.8921348314606742, 0.8921348314606742, 0.9056179775280899, 0.9056179775280899]


In [16]:
print(max(accuracy_list))
print(max_C,max_k)

0.9056179775280899
296 rbf


In [18]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8853932584269663


In [19]:
def predict_category(text):
    text_tf = vectorizer.transform([text])
    
    topic_dist = lda.transform(text_tf)
    
    category = svm.predict(topic_dist)
    
    return category



In [20]:
text1 = "Britain's stock market has lost its position as Europe's most-valued, with France taking the top spot, data shows.A weak pound, fears of recession in the UK and surging sales at French luxury goods makers are thought to be behind the shift, according to data from Bloomberg."
text2 = "The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL."
text3 = "The government just announced a new policy on immigration."
text4  = "Piero, named after the Italian painter and pioneer of perspective Piero della Francesco, creates a virtual stadium in which virtual players can be tracked from almost any angle. Viewers will be able to see precisely how the ball was thrown and by whom, giving a greater depth to the growing wealth of analysis available during sports broadcasts. The technology has been created by BBC Research and Development for BBC Broadcast and BBC Outside Broadcasts. BBC Sport is the first client to start using the system.   In order to keep audiences growing and growing... we need to work closely with people who create technology and innovation to bring sport to life,   said Andrew Thompson, the BBC   s head of development, new media and sports news.   We want to appeal to core fans - to give them more analysis, more detail, more definitive answers about key passages of play.   Piero works by taking telemetric data from fixed camera positions and sending that data inside the video signal to a PC which can then render the information into 3D graphics."
text5 = "International films will be given the same prominence as US films at next years Sundance Film Festival, with movies dominated by the theme of war. The independent film festival will feature two new international cinema competitions, during its 20-30 January season in Utah. Forty-two films will debut at Sundance, including The Liberace of Baghdad by British director Sean McAllister. The prestigious festival was founded by actor Robert Redford in 1981. We have always had an international component, but from next year they will enter a jury competition, festival director Geoffrey Gilmore said. We wanted to give world cinema more emphasis and have now put it on par with the American dramatic and documentary competitions. Twelve films competing in the new world cinema documentary category focus on countries and people under siege."

print(predict_category(text1))  # Output: business
print(predict_category(text2))  # Output: entertainment
print(predict_category(text3))  # Output: politics
print(predict_category(text4))
print(predict_category(text5))

['entertainment']
['business']
['politics']
['tech']
['entertainment']


In [22]:
text1 = "Apple is expected to release a new iPhone next month"
text2 = "The stock market had a strong performance last quarter"
text3 = "World Cup 2026: Fifa set to make decision on four-team group format this month"
text4 = "Walker had hoped to compete in the New Year Sprint which is staged at Musselburgh Racecourse near Edinburgh on Tuesday and Wednesday. The 31-year-old Scot was suspended for two years in 1998 after testing positive for nandrolone. I had intended to race but Im running like a goon, said Walker. He told the Herald newspaper: Im not in great shape, after missing about a month of training."
text5 = "With unemployment stuck above 4 million for years, the government of Chancellor Gerhard Schroeder has put job creation at the top of the agenda. A controversial package of measures to shake up incentives to get back to work, paid for by cutting some cherished benefits, has sparked anger among some German workers. Strikes in a number of industries, notably among the country's iconic carmakers, have demonstrated the displeasure - as well as fears about further job losses as outsourcing takes hold."
text6 = "The Bedford star returned to London 12 months later, lowering her mixed-race world record of 2:17:18, which she set in Chicago in October 2003, by one minute 53 secs. Radcliffe's career took a setback when she failed to complete the Olympic marathon and later dropped out of the Athens 10,000m last August. But the 31-year-old bounced back to win the New York Marathon in November."
text7 = "It is thought the proposed overhaul, due on Thursday, could mean pensions could be based on a career average salary. For each year served, staff currently get one eightieth of their highest salary in the final three years. Ministers will be anxious to avoid mass strike action in the lead-up to the next general election, which is widely expected next May. In a statement on Sunday, the Cabinet Office said it was reviewing the Civil Service Pension Scheme, and hoped to announce proposals soon."
text8 = "Film-maker Spike Lee says black representation is stronger than ever in cinema and TV but the true power in entertainment lies behind the camera. The She Hate Me director urged students at his old Atlanta university, Morehouse College, to seek gatekeeper positions behind the scenes. Lee told them to work up the corporate ladder because everybody cannot be an actor, everybody cannot make a record. He spoke as part of a discussion panel, then led a retrospective of his films. Returning to his old university, which educates only African American students, Lee discussed the challenges facing black people in the entertainment industry. Even Denzel (Washington), he is getting $20m a movie. But when it comes time to do a movie, he has to go to one of those gatekeepers,Lee said. He told aspiring young film-makers in the audience not to ignore non-traditional routes to getting a movie made, including raising funds independently and releasing films straight to DVD."

print("text1",predict_category(text1))  # Output: business
print("text2",predict_category(text2))  # Output: entertainment
print("text3",predict_category(text3))  # Output: politics
print("text4",predict_category(text4))
print("text5",predict_category(text5))
print("text6",predict_category(text6))
print("text7",predict_category(text7))
print("text8",predict_category(text8))
# docs = [text1, text2, text3]

# print(predict_category(docs)) 


text1 ['entertainment']
text2 ['entertainment']
text3 ['sport']
text4 ['sport']
text5 ['business']
text6 ['sport']
text7 ['politics']
text8 ['entertainment']


In [23]:
text9 = "The Commission for Africa s report will be released on 11 March - Comic Relief day, Tony Blair has said. Julys G8 summit in Gleneagles in Perthshire chaired by the prime minister will use the report as the basis for talks on Africa. The announcement followed the final meeting of the commission which includes singer Bob Geldof in London. As well as more aid, fairer trade and less debt, the commission is likely to demand action on corruption in Africa. Mr Blair told a news conference: It will be a report that is brutally frank about the reality, but I hope idealistic about what can be done if the will is there. It is an ambitious project we have set ourselves and you will have to judge on its outcome when we publish it."
text10 = "Dutch watch Van Gogh is last film The last film to be made by the slain Dutch director Theo van Gogh, called 06/05, has been premiered in The Hague. Members of Van Gogh is family and celebrities attended the screening of 06/05, based on the murder of the anti-immigration politician Pim Fortuyn. Van Gogh was shot and stabbed to death in Amsterdam six weeks ago. A Dutch-Moroccan man suspected of radical Islamist links has been charged. The killing, and Fortuyn is death in 2002, convulsed the Netherlands. Many Dutch people have questioned their country is self-image as a peaceful, tolerant nation in the wake of the murders, which triggered heated debate about immigration."

print("text9", predict_category(text9))
print("text10", predict_category(text10))

text9 ['politics']
text10 ['politics']


In [24]:
text_new = "Indie film nominations announced Mike Leighs award-winning abortion drama Vera Drake has scooped seven nominations at this years British Independent Film Awards.But the Venice winner faces stiff competition from Shane Meadows critically acclaimed Dead Mans Shoes, which received eight nominations. Also in the running for a clutch of awards are My Summer of Love and the stalker drama Enduring Love. The winners will be announced at a ceremony in London on 30 November. The winners of the awards will be chosen by a jury chaired by Cold Mountain director Anthony Minghella and including actresses Cate Blanchett and Helena Bonham-Carter. The awards, which recognise independent film-making in Britain, were established seven years ago. This years nominees reflect the growing strength and diversity of British independent filmmaking, said BIFA founder and director Elliot Grove. Commenting on the diversity of the nominated films, he added: Our selection committee had a harder time than ever narrowing down the field."
print(predict_category(text_new))

['entertainment']
